In [ ]:
import tensorflow as tf
import numpy as np


In [ ]:
char_rdic = ['h', 'e', 'l', 'o']
char_dic = {w: i for i, w in enumerate(char_rdic)}
x_data = np.array([ [1, 0, 0, 0],
                  [0, 1, 0, 0],
                  [0, 0, 1, 0], 
                  [0, 0, 1, 0]], dtype='f')
sample = [char_dic[c] for c in 'hello']

In [ ]:
char_vocab_size = len(char_dic)
rnn_size = char_vocab_size # 1 hot coding. one of 4
time_step_size = 4 # 'hell' -> predict 'ello'
batch_size = 1 # one sample

In [ ]:
rnn_cell = tf.contrib.rnn.BasicRNNCell(num_units = rnn_size)
#rnn_cell = tf.contrib.rnn.BasicLSTMCell(rnn_size)
#rnn_cell = tf.contrib.rnn.GRUCell(rnn_size)
state = tf.zeros([batch_size, rnn_cell.state_size])
X_split = tf.split(x_data, time_step_size, axis = 0)
outputs, state = rnn_cell(X_split, state)

# rnn_size = output의 vector size. num of classes(characters). 글자가 네종류면 4. 
# rnn의 time 갯수는 실행시킬 때 결정할 수 있다. 다섯 글자를 넣고 싶으면 다섯개를.
# training 결과로 state 값이 만들어지는데 나중에 학습할 때 이 state를 다시 사용할 수 있다.

In [ ]:
print(rnn_cell.state_size)
print(rnn_cell.output_size)
print(rnn_cell.zero_state)

In [ ]:
logits = tf.reshape(tf.concat(1, outputs), [-1, rnn_size])
targets = tf.reshape(sample[1:], [-1])
weights = tf.ones([time_step_size * batch_size])

In [ ]:
print(outputs)
print(logits.get_shape())
print(targets.get_shape())
print(weights.get_shape())

In [ ]:
loss = tf.nn.seq2seq.sequence_loss_by_example([logits], [targets], [weights])
cost = tf.reduce_sum(loss)/batch_size
train_op = tf.train.RMSPropOptimizer(0.01, 0.9).minimize(cost)

In [ ]:
sess = tf.InteractiveSession()
sess.run(tf.initialize_all_variables())

In [ ]:
for i in range(100):
    sess.run(train_op)
    result = sess.run(tf.arg_max(logits, 1))
    print(sess.run(outputs))
    print(sess.run(state))
    print(result, [char_rdic[t] for t in result])
    print('----')

In [ ]:
sess.close()